In [ ]:
!pip install annoy
import numpy as np
from annoy import AnnoyIndex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 KB 40.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.1-cp39-cp39-linux_x86_64.whl size=582312 sha256=e6294ad65a8f076701d33bf0df315ac011cff12cf31cc461e575f15aaba5dbb2
  Stored in directory: /root/.cache/pip/wheels/5b/7d/31/9a9a4993d085bc85bee21946bce94cd5906ce99730f5467e57
Successfully built annoy


In [ ]:
class PreTrainedEmbeddings(object):
  def __init__(self,word_to_index,word_vectors):
    '''
    Args :
    word_to_index (dict): mapping from word to integers
    word_vectors (list of numpy arrays)
    '''

    self.word_to_index = word_to_index
    self.word_vectors = word_vectors
    print(self.word_vectors)
    self.index_to_word = {i:w for w, i in self.word_to_index.items()}
    print(self.word_to_index)
    print(self.index_to_word)
    self.index = AnnoyIndex(len(word_vectors[0]), metric='euclidean')
    for _, i in self.word_to_index.items():
      print(i) #word_to_index
      self.index.add_item(i, self.word_vectors[i])  #(0,word_vect[0])
    self.index.build(50) 
    print(self.index)
    
  # def get_nearest_neighbors(self, word, k=10):
  #       if word not in self.word_to_index:
  #           return []
  #       word_index = self.word_to_index[word]
  #       vector = self.word_vectors[word_index]
  #       nn_indices = self.index.get_nns_by_vector(vector, k)
  #       return [self.index_to_word[i] for i in nn_indices]   
  
  @classmethod
  def from_embeddings_file(cls,embedding_file):
    
    """
    Instantiate from pretrained vector file.
    Vector file should be of the format:
    word0 x0_0 x0_1 x0_2 x0_3 ... x0_N
    word1 x1_0 x1_1 x1_2 x1_3 ... x1_N
    Args:
    embedding_file (str): location of the file
    Returns:
    instance of PretrainedEmbeddings
    """
    word_to_index= {}
    word_vectors = []
    with open(embedding_file) as fp:
      for line in fp.readlines():
        line = line.split(' ')
        word = line[0]                      #'apple' 1,2,3,4,5,6
        vec = np.array([float(x) for x in line[1:]])
        word_to_index[word] = len(word_to_index)
        word_vectors.append(vec)
    return cls(word_to_index , word_vectors)   



In [ ]:
word_to_index = {'apple': 0, 'banana': 1, 'orange': 2, 'pear': 3}
word_vectors = [np.array([1.0, 2.0, 3.0]), np.array([4.0, 5.0, 6.0]), np.array([7.0, 8.0, 9.0]), np.array([10.0, 11.0, 12.0])]
index = PreTrainedEmbeddings(word_to_index, word_vectors)


TypeError: ignored

In [ ]:
glove_file = '/content/drive/MyDrive/glove.6B.50d.txt'
embeddings = PreTrainedEmbeddings.from_embeddings_file(glove_file)

KeyboardInterrupt: ignored

In [ ]:
len(embeddings.word_vectors)

400000

In [ ]:
len(embeddings.word_vectors[0])

50

Get the neighbors nad compute arthmetics on vector

In [ ]:
class PreTrainedEmbeddings(object):
  def __init__(self,word_to_index,word_vectors):
    '''
    Args :
    word_to_index (dict): mapping from word to integers
    word_vectors (list of numpy arrays)
    '''

    self.word_to_index = word_to_index
    self.word_vectors = word_vectors
    #print(self.word_vectors)
    self.index_to_word = {i:w for w, i in self.word_to_index.items()}
    #print(self.word_to_index)
    #print(self.index_to_word)
    self.index = AnnoyIndex(len(word_vectors[0]), metric='euclidean')
    for _, i in self.word_to_index.items():
      #print(i) #word_to_index
      self.index.add_item(i, self.word_vectors[i])  #(0,word_vect[0])
    self.index.build(50) 
    print(self.index)
    
  # def get_nearest_neighbors(self, word, k=10):
  #       if word not in self.word_to_index:
  #           return []
  #       word_index = self.word_to_index[word]
  #       vector = self.word_vectors[word_index]
  #       nn_indices = self.index.get_nns_by_vector(vector, k)
  #       return [self.index_to_word[i] for i in nn_indices]   
  
  @classmethod
  def from_embeddings_file(cls,embedding_file):
    
    """
    Instantiate from pretrained vector file.
    Vector file should be of the format:
    word0 x0_0 x0_1 x0_2 x0_3 ... x0_N
    word1 x1_0 x1_1 x1_2 x1_3 ... x1_N
    Args:
    embedding_file (str): location of the file
    Returns:
    instance of PretrainedEmbeddings
    """
    word_to_index= {}
    word_vectors = []
    with open(embedding_file) as fp:
      for line in fp.readlines():
        line = line.split(' ')
        word = line[0]                      #'apple' 1,2,3,4,5,6
        vec = np.array([float(x) for x in line[1:]])
        word_to_index[word] = len(word_to_index)
        word_vectors.append(vec)
    return cls(word_to_index , word_vectors)   

  def get_embeddings(self,word):
    return self.word_vectors[self.word_to_index[word]]  

  def get_closest_to_vectors(self,vector,n=1):
    nn_indices = self.index.get_nns_by_vector(vector,n)
    #print(nn_indices) #gives a index of word in decreasing order of their distance to target word
    return [self.index_to_word[neighbor] for neighbor in nn_indices]

  def compute_and_print_analogy(self, word1, word2, word3):
    """Prints the solutions to analogies using word embeddings
    Analogies are word1 is to word2 as word3 is to __
    This method will print: word1 : word2 :: word3 : word4
    Args:
    word1 (str)
    word2 (str)
    word3 (str)
    """
    vec1 = self.get_embeddings(word1)
    vec2 = self.get_embeddings(word2)
    vec3 = self.get_embeddings(word3)
    spatial_relationship = vec2 - vec1
    vec4 = vec3 + spatial_relationship

    closest_words = self.get_closest_to_vectors(vec4, n=4)
    existing_words = set([word1, word2, word3])
    closest_words = [word for word in closest_words if word not in existing_words]
    if len(closest_words) == 0:
      print("Could not find nearest neighbors for the vector!")
      return
    for word4 in closest_words:
      print("{} : {} :: {} : {}".format(word1, word2, word3, word4))



In [ ]:
# #example
# word_to_index = {'apple': 0, 'banana': 1, 'orange': 2, 'pear': 3}
# word_vectors = [np.array([1.0, 2.0, 3.0]), np.array([4.0, 5.0, 6.0]), np.array([7.0, 8.0, 9.0]), np.array([10.0, 11.0, 12.0])]
# index = AnnoyIndex(len(word_vectors[0]), metric='euclidean')
# for _, i in word_to_index.items():
#   print(i) #word_to_index
#   index.add_item(i, word_vectors[i])  #(0,word_vect[0])
# index.build(50) 


0
1
2
3


True

In [ ]:
# index.get_nns_by_vector(np.array([11,22,23]),3) #gives 3 utputs in decreasing order of distance

[3, 2, 1]

In [ ]:
glove_file = '/content/drive/MyDrive/glove.6B.50d.txt'
embeddings = PreTrainedEmbeddings.from_embeddings_file(glove_file)

In [ ]:
embeddings.get_embeddings('apple')

array([ 0.52042 , -0.8314  ,  0.49961 ,  1.2893  ,  0.1151  ,  0.057521,
       -1.3753  , -0.97313 ,  0.18346 ,  0.47672 , -0.15112 ,  0.35532 ,
        0.25912 , -0.77857 ,  0.52181 ,  0.47695 , -1.4251  ,  0.858   ,
        0.59821 , -1.0903  ,  0.33574 , -0.60891 ,  0.41742 ,  0.21569 ,
       -0.07417 , -0.5822  , -0.4502  ,  0.17253 ,  0.16448 , -0.38413 ,
        2.3283  , -0.66682 , -0.58181 ,  0.74389 ,  0.095015, -0.47865 ,
       -0.84591 ,  0.38704 ,  0.23693 , -1.5523  ,  0.64802 , -0.16521 ,
       -1.4719  , -0.16224 ,  0.79857 ,  0.97391 ,  0.40027 , -0.21912 ,
       -0.30938 ,  0.26581 ])

In [ ]:
embeddings.compute_and_print_analogy('apple', 'banana', 'pear')

apple : banana :: pear : avocado
apple : banana :: pear : seaweed
apple : banana :: pear : beet


In [ ]:
embeddings.compute_and_print_analogy('man', 'king', 'woman')

man : king :: woman : queen
man : king :: woman : prince
man : king :: woman : daughter


In [ ]:
embeddings.compute_and_print_analogy('man', 'doctor', 'woman')
#biases in embeddings

man : doctor :: woman : nurse
man : doctor :: woman : pregnant
man : doctor :: woman : child
